In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
from torch.nn import functional as F
import torchvision
from torchvision import datasets,models,transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import time
import copy

plt.ion()

In [ ]:
torch.cuda.is_available()

In [ ]:
transforms=transforms.Compose([transforms.Resize(226),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [ ]:
image_dataset=datasets.ImageFolder('test',transform=transforms)
print ("Length of image_dataset is {}".format(len(image_dataset)))

In [ ]:
dataloaders={'testing':torch.utils.data.DataLoader(image_dataset, batch_size=4,
                                             shuffle=True, num_workers=4)}

In [ ]:
class_names = image_dataset.classes
class_names

In [ ]:
device = torch.device('cpu')

print(device)

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['testing']))
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
images=[]
def visualize_model(model, num_images=48):
    was_training = model.training
    model.eval()
    images_so_far = 0
    
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['testing']):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                if (class_names[preds[j]] != class_names[labels[j]]):
                    images.append(inputs[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
model=torch.load('myriadcara.pt')
model.training

In [ ]:
visualize_model(model)

In [ ]:
print (len(images))

In [ ]:
fig=plt.figure(figsize=(10,10))
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
for i,img in enumerate(images):
    image=(img.numpy().transpose(1,2,0))*std+mean
    image=np.clip(image,0,1)
    fig.add_subplot(3,4,i+1)
    plt.imshow(image)
plt.show()